# Geographic Visualization

## This script contains the following points:
### 01. Importing libraries
### 02. Importing Data
### 03. Geographical Visualizations

## 01. Importing libraries

In [17]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import os
import folium
from folium import plugins
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap
import json
import geopandas as gpd
import pyproj
from pyproj import CRS, Transformer

## 02. Importing Data

#### Note: Source two files - one standard data set ('df_failures') and one JSON/GeoJSON (' Schadensdaten_geo') that complement each other with a common column of location data ('id_failure').

In [18]:
# Define path
path = r'C:\Users\Sanja\Documents\11-2023 Water Loss Project'

In [19]:
#Import the standard dataset 'pipes_failures_merged_all'
df_failures = pd.read_excel(os.path.join(path, '02 Data', 'Prepared Data', 'failures_clean.xlsx'), index_col = False)

In [20]:
df_failures.head(5)

,Unnamed: 0,id_failure,id_pipeline,failure_reported_on,cause_of_damage
0,0,1196,2067243,1981-11-14,NaN
1,1,1443,2074816,1998-11-12,NaN
2,2,83500002/15,2068165,2006-07-27,NaN
3,3,1433,2062991,1998-01-13,NaN
4,4,1990,2056069,2004-08-10,NaN


### Import the Geo Dataset

#### Note: The geodataset is an excel file 'Schadensdaten_geo' where the coordinates are given in a projected coordinate system (UTM Zone 32N), so we need to covert them to latitude and longitude. We can use  the geopandas library for spatial data handling and the pyproj.CRS class for CRS definitions. 

#### a. Convert the coordinates X and Y from UTM Zone 32N to to latitude and longitude

In [21]:
# Read the Excel file
excel_file_path = os.path.join(path, '02 Data', 'Original Data', 'Schadensdaten_geo.xlsx')
df = pd.read_excel(excel_file_path)

# Convert DataFrame to GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['X'], df['Y']))

# Define the input CRS 
input_crs = CRS('EPSG:32632')  # UTM Zone 32N

# Define the output CRS (standard geographic coordinate system - WGS 84)
output_crs = CRS('EPSG:4326')

# Create a PyProj transformer
transformer = Transformer.from_crs(input_crs, output_crs, always_xy=True)

# Transform X and Y coordinates to latitude and longitude
gdf = gdf.set_crs(input_crs)
gdf = gdf.to_crs(output_crs)

# Save the transformed GeoDataFrame back to Excel
output_excel_file_path = os.path.join(path, '02 Data', 'Prepared Data', 'Schadensdaten_geo_transformed.xlsx')
gdf.to_excel(output_excel_file_path, index=False)

# Display the transformed GeoDataFrame
print(gdf)

         id_failure   Gemeinde            Straße     ENum           Y  \
0              1070  Pforzheim     Belfortstraße  1656709  5417392.97   
1              2068  Pforzheim     Belfortstraße  1753200  5417429.32   
2     82300002/1007  Pforzheim     Belfortstraße  1820500  5417296.47   
3              1001  Pforzheim      Belremstraße  1769375  5414689.30   
4              1681  Pforzheim      Belremstraße  1656724  5414577.74   
...             ...        ...               ...      ...         ...   
1459   82300002/903  Pforzheim  Zum Geigersgrund  1791217  5418493.87   
1460           1820  Pforzheim  Zum Höhenfreibad  1652716  5418663.18   
1461           1821  Pforzheim  Zum Höhenfreibad  1652718  5418661.76   
1462   82300002/906  Pforzheim  Zum Höhenfreibad  1808207  5418641.87   
1463  82300002/1091  Pforzheim  Zum Lachenwäldle  1885987  5416400.07   

              X                  geometry  
0     477349.61  POINT (8.69089 48.90906)  
1     477353.40  POINT (8.69094 48.

In [22]:
# Read the Excel file
excel_file_path = os.path.join(path, '02 Data', 'Original Data', 'Schadensdaten_geo.xlsx')
df = pd.read_excel(excel_file_path)

# Convert DataFrame to GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['X'], df['Y']))

# Define the input CRS 
input_crs = CRS('EPSG:32632')  # UTM Zone 32N

# Set the CRS for the GeoDataFrame
gdf = gdf.set_crs(input_crs)

# Define the output CRS (standard geographic coordinate system - WGS 84)
output_crs = CRS('EPSG:4326')

# Create a PyProj transformer
transformer = Transformer.from_crs(input_crs, output_crs, always_xy=True)

# Transform X and Y coordinates to latitude and longitude
gdf['Longitude'], gdf['Latitude'] = transformer.transform(gdf['geometry'].x, gdf['geometry'].y)

# Save the transformed GeoDataFrame back to Excel
output_excel_file_path = os.path.join(path, '02 Data', 'Prepared Data', 'Schadensdaten_geo_transformed.xlsx')
gdf.to_excel(output_excel_file_path, index=False)

# Display the transformed GeoDataFrame
print(gdf)

         id_failure   Gemeinde            Straße     ENum           Y  \
0              1070  Pforzheim     Belfortstraße  1656709  5417392.97   
1              2068  Pforzheim     Belfortstraße  1753200  5417429.32   
2     82300002/1007  Pforzheim     Belfortstraße  1820500  5417296.47   
3              1001  Pforzheim      Belremstraße  1769375  5414689.30   
4              1681  Pforzheim      Belremstraße  1656724  5414577.74   
...             ...        ...               ...      ...         ...   
1459   82300002/903  Pforzheim  Zum Geigersgrund  1791217  5418493.87   
1460           1820  Pforzheim  Zum Höhenfreibad  1652716  5418663.18   
1461           1821  Pforzheim  Zum Höhenfreibad  1652718  5418661.76   
1462   82300002/906  Pforzheim  Zum Höhenfreibad  1808207  5418641.87   
1463  82300002/1091  Pforzheim  Zum Lachenwäldle  1885987  5416400.07   

              X                        geometry  Longitude   Latitude  
0     477349.61  POINT (477349.610 5417392.970)   8

#### b. Covert the transformed excel file 'Schadensdaten_geo_transformed.xlsx' to GeoJSON file

In [23]:
# Read Excel file into a DataFrame
df = pd.read_excel(os.path.join(path, '02 Data', 'Prepared Data', 'Schadensdaten_geo_transformed.xlsx'))

# Create a GeoDataFrame from the DataFrame
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['Longitude'], df['Latitude']))

# Specify the full path for the output GeoJSON file
output_path = os.path.join(path, '02 Data', 'Prepared Data', 'Schadensdaten_geo.geojson')

# Write GeoJSON data to the specified file
gdf.to_file(output_path, driver='GeoJSON')

In [24]:
# Import "geojson" file
failuers_geo = gpd.read_file(os.path.join(path, '02 Data', 'Prepared Data', 'Schadensdaten_geo.geojson'))

In [25]:
failuers_geo

,id_failure,Gemeinde,Straße,ENum,Y,X,Longitude,Latitude,geometry
0,1070,Pforzheim,Belfortstraße,1656709,5417392.97,477349.61,8.690887,48.909064,POINT (8.69089 48.90906)
1,2068,Pforzheim,Belfortstraße,1753200,5417429.32,477353.40,8.690937,48.909391,POINT (8.69094 48.90939)
2,82300002/1007,Pforzheim,Belfortstraße,1820500,5417296.47,477308.70,8.690335,48.908194,POINT (8.69033 48.90819)
3,1001,Pforzheim,Belremstraße,1769375,5414689.30,476864.80,8.684424,48.884725,POINT (8.68442 48.88472)
4,1681,Pforzheim,Belremstraße,1656724,5414577.74,476706.49,8.682271,48.883715,POINT (8.68227 48.88372)
...,...,...,...,...,...,...,...,...,...
1459,82300002/903,Pforzheim,Zum Geigersgrund,1791217,5418493.87,476564.60,8.680111,48.918938,POINT (8.68011 48.91894)
1460,1820,Pforzheim,Zum Höhenfreibad,1652716,5418663.18,479206.14,8.716159,48.920555,POINT (8.71616 48.92056)
1461,1821,Pforzheim,Zum Höhenfreibad,1652718,5418661.76,479207.73,8.716180,48.920543,POINT (8.71618 48.92054)
1462,82300002/906,Pforzheim,Zum Höhenfreibad,1808207,5418641.87,479196.31,8.716026,48.920363,POINT (8.71603 48.92036)


### 03. Geographical Visualizations

In [26]:
# Create a data frame with just the id_failure and the values for failures we want plotted

data_to_plot = df_failures[['id_failure','cause_of_damage']]
data_to_plot.head()

,id_failure,cause_of_damage
0,1196,NaN
1,1443,NaN
2,83500002/15,NaN
3,1433,NaN
4,1990,NaN


In [27]:
# Open and read the GeoJSON file
with open(r'C:\Users\Sanja\Documents\11-2023 Water Loss Project\02 Data\Prepared Data\Schadensdaten_geo.geojson', 'r') as f:
    # Load the JSON data
    data = json.load(f)

# Iterating through the list of features
for feature in data['features']:
    # Print the properties of each feature
    print("Feature Properties:")
    for key, value in feature['properties'].items():
        print(f"  {key}: {value}")

    # Print the geometry of each feature
    print("Feature Geometry:")
    print(f"  Type: {feature['geometry']['type']}")
    print(f"  Coordinates: {feature['geometry']['coordinates']}")

    print("\n" + "-" * 30 + "\n")  # Separating features for better readability

Feature Properties:
  id_failure: 1070
  Gemeinde: Pforzheim
  StraÃŸe: BelfortstraÃŸe
  ENum: 1656709
  Y: 5417392.97
  X: 477349.61
  Longitude: 8.690887438392448
  Latitude: 48.90906384828052
Feature Geometry:
  Type: Point
  Coordinates: [8.690887438392448, 48.90906384828052]

------------------------------

Feature Properties:
  id_failure: 2068
  Gemeinde: Pforzheim
  StraÃŸe: BelfortstraÃŸe
  ENum: 1753200
  Y: 5417429.32
  X: 477353.4
  Longitude: 8.690937143454097
  Latitude: 48.90939097796691
Feature Geometry:
  Type: Point
  Coordinates: [8.690937143454097, 48.90939097796691]

------------------------------

Feature Properties:
  id_failure: 82300002/1007
  Gemeinde: Pforzheim
  StraÃŸe: BelfortstraÃŸe
  ENum: 1820500
  Y: 5417296.47
  X: 477308.7
  Longitude: 8.690334507586583
  Latitude: 48.90819427243528
Feature Geometry:
  Type: Point
  Coordinates: [8.690334507586583, 48.90819427243528]

------------------------------

Feature Properties:
  id_failure: 1001
  Gemeinde: 

### a. Plotting a map with clustered markers, providing a clear representation of the failures (point) data, using the MarkerCluster plugin from the Folium library

In [28]:
# Create a folium map centered on the average latitude and longitude
map_center = [failuers_geo['Latitude'].mean(), failuers_geo['Longitude'].mean()]
map = folium.Map(location=map_center, zoom_start=10)

# Add MarkerCluster to the map
marker_cluster = MarkerCluster().add_to(map)

# Add markers to the MarkerCluster
for idx, row in failuers_geo.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"ID: {row['id_failure']}",
    ).add_to(marker_cluster)

# Display the map in the notebook
display(map)

In [29]:
# Save the map as an HTML file
map_output_path = r'C:\Users\Sanja\Documents\11-2023 Water Loss Project\04 Analysis\Visualizations\cluster_failures.html'
map.save(map_output_path)

### b. Plotting a heatmap that visually represents the density of failure points on the map, using the HeatMap plugin from the Folium library¶

In [30]:
# Create a list to store the coordinates for the heatmap
heatmap_data = []

# Iterate through the GeoJSON features
for feature in data['features']:
    # Append the coordinates to the heatmap_data list
    heatmap_data.append([feature['geometry']['coordinates'][1], feature['geometry']['coordinates'][0]])

# Setup a folium map at a high-level zoom
map_center = [data['features'][0]['geometry']['coordinates'][1], data['features'][0]['geometry']['coordinates'][0]]
map = folium.Map(location=map_center, zoom_start=10)

# Add a HeatMap layer to the map using the heatmap_data
HeatMap(heatmap_data).add_to(map)

# Custom legend HTML and CSS
legend_html = '''
     <div style="position: fixed; bottom: 50px; left: 50px; z-index:1000; background-color:white; padding: 10px; border:1px solid grey; font-size:14px;">
     <p><strong>Legend</strong></p>
     <p><span style="background-color: #0000FF; padding: 5px;"></span> Low Density</p>
     <p><span style="background-color: #00FF00; padding: 5px;"></span> Medium Density</p>
     <p><span style="background-color: #FF0000; padding: 5px;"></span> High Density</p>
     <!-- Add more lines as needed -->
     </div>
'''

# Display the custom legend
map.get_root().html.add_child(folium.Element(legend_html))

# Display the map in the notebook
display(map)

In [31]:
# Save the map as an HTML file
map_output_path = r'C:\Users\Sanja\Documents\11-2023 Water Loss Project\04 Analysis\Visualizations\heatmap_failures.html'
map.save(map_output_path)

### c. Plotting a marker map that represents the failures causes on the map, using the folium.Marker class 

#### Merge DataFrames:

In [32]:
# Merge the two dataframes: GeoDataFrame 'failuers_geo' and the dataframe 'data_to_plot'
merged_geo_df = pd.merge(failuers_geo, data_to_plot, on='id_failure', how='inner')

In [33]:
merged_geo_df

,id_failure,Gemeinde,Straße,ENum,Y,X,Longitude,Latitude,geometry,cause_of_damage
0,82300002/1007,Pforzheim,Belfortstraße,1820500,5417296.47,477308.70,8.690335,48.908194,POINT (8.69033 48.90819),3.0
1,82300002/1175,Pforzheim,Belremstraße,1908046,5413979.82,476072.08,8.673653,48.878312,POINT (8.67365 48.87831),2.0
2,82300002/1375,Pforzheim,Birkenweg,1976665,5416259.27,479964.30,8.726626,48.898956,POINT (8.72663 48.89896),3.0
3,82300002/1480,Pforzheim,Bleichstraße,2010565,5416164.71,477749.58,8.696413,48.898029,POINT (8.69641 48.89803),2.0
4,82300002/999,Pforzheim,Bleichstraße,1854354,5416621.20,477793.89,8.696992,48.902137,POINT (8.69699 48.90214),2.0
...,...,...,...,...,...,...,...,...,...,...
463,82300002/1450,Pforzheim,Zerrennerstraße,1998230,5417099.67,477567.42,8.693876,48.906433,POINT (8.69388 48.90643),3.0
464,82300002/901,Pforzheim,Zerrennerstraße,1807950,5417109.58,477581.63,8.694069,48.906523,POINT (8.69407 48.90652),3.0
465,82300002/903,Pforzheim,Zum Geigersgrund,1791217,5418493.87,476564.60,8.680111,48.918938,POINT (8.68011 48.91894),0.0
466,82300002/906,Pforzheim,Zum Höhenfreibad,1808207,5418641.87,479196.31,8.716026,48.920363,POINT (8.71603 48.92036),2.0


#### Map Causes:

In [34]:
# Map the causes to the GeoJSON features
failuers_geo['cause_of_damage'] = merged_geo_df['cause_of_damage']

In [38]:
# Create a map centered around the mean coordinates of your GeoDataFrame
mean_lat = merged_geo_df['Latitude'].mean()
mean_lon = merged_geo_df['Longitude'].mean()

# Create a color dictionary for different causes
cause_colors = {
    '2.0': 'red',
    '3.0': 'blue',
    '6.0': 'gray',
    '1.0': 'gray',
}


# Convert keys to strings to ensure consistency
cause_colors = {str(key): value for key, value in cause_colors.items()}

# Default color for unknown causes
default_color = 'gray'

# Ensure default_color is a string
default_color = str(default_color)

map_causes = folium.Map(location=[mean_lat, mean_lon], zoom_start=12)

# Create a FeatureGroup for each cause color
cause_groups = {color: plugins.FeatureGroupSubGroup(map_causes, color) for color in cause_colors.values()}
cause_groups[default_color] = plugins.FeatureGroupSubGroup(map_causes, default_color)  # Add FeatureGroup for default color

# Iterate through GeoDataFrame rows and add markers to the map
for idx, row in merged_geo_df.iterrows():
    cause = str(row['cause_of_damage'])
    
    # Check for NaN values and assign a default color
    if pd.isna(cause):
        color = default_color
    else:
        # Assign a default color for unknown causes
        color = cause_colors.get(cause, default_color)
    
    folium.Marker([row['Latitude'], row['Longitude']], popup=cause, icon=folium.Icon(color=color)).add_to(cause_groups[color])

# Add each FeatureGroup to the map
for cause, group in cause_groups.items():
    map_causes.add_child(group)
    map_causes.get_root().add_child(folium.Element(f'<p style="color:{cause}">{cause}</p>'))

# Add a Legend
legend_html = '''
     <div style="position: fixed; 
                 bottom: 50px; left: 50px; width: 150px; height: 90px; 
                 border:2px solid grey; z-index:9999; font-size:14px;
                 background-color:white;
                 ">&nbsp; Cause Legend <br>
                 &nbsp; Code: 2.0 &nbsp; <i class="fa fa-circle fa-1x" style="color:red"></i><br>
                 &nbsp; Code: 3.0 &nbsp; <i class="fa fa-circle fa-1x" style="color:blue"></i><br>
                 &nbsp; Other &nbsp; <i class="fa fa-circle fa-1x" style="color:gray"></i>
      </div>
     '''

map_causes.get_root().html.add_child(folium.Element(legend_html))

# Add a LayerControl to the map
folium.LayerControl(collapsed=False).add_to(map_causes)

# Display the map in the notebook
display(map_causes)

In [36]:
# Save the map as an HTML file
map_causes_output_path = r'C:\Users\Sanja\Documents\11-2023 Water Loss Project\04 Analysis\Visualizations\map_failures_causes.html'
map.save(map_causes_output_path)

### Observation: 
I've created three types of maps:
#### 1. Folium map with clustered markers:
- Provides a clear representation of the locations of failures (points) on the map.
- Clustering helps in visualizing densely populated areas with a large number of failure points.
- Allows for an overview of the geographical distribution of failures.
#### 2. Heatmap that visually represents the density of failure points on the map
- Visually represents the density of failure points on the map.
- Highlights areas with higher concentrations of failures.
#### 3. Map with markers representing points from a GeoDataFrame, with each marker color-coded based on a cause of damage.
- Provides insights into the causes of failures by color-coding markers.
- Allows for the identification of specific causes in different geographical areas.
### Overall Observations:
The combination of the three maps offers a comprehensive view of the failure data from different perspectives.
The clustered marker map and heatmap focus on the geographical distribution and density of failures, while the color-coded map provides insights into the causes of these failures.
These visualizations can aid in identifying patterns, hotspots, and potential correlations between geographical locations and causes of failures.
### Research Questions:
#### Existing Research Questions:
The maps provide a spatial understanding of failure occurrences, but additional analysis is required to answer specific research questions.
Consideration of temporal aspects could enhance the analysis (e.g., changes in failure patterns over time).
#### New Research Questions:
Are there specific clusters of failures that coincide with particular causes?
How has the distribution of failures changed over time, if applicable?
Are there any geographical or environmental factors contributing to the observed patterns?
####  Possibilities for testing hypotheses based on the maps and data visualizations:
##### Hypothesis 1: There is a spatial correlation between the geographical distribution of failure clusters and specific environmental factors.
##### Hypothesis 2: Failures with similar causes tend to cluster in specific geographical regions.
##### Hypothesis 3: The frequency and causes of failures vary over time.